## Settings and pointers

In [28]:
service_account_file = '../../creds/google__sa.json'

tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
included_sheet_names = [
    "Conversations_Batch_7",
    "Conversations_Batch_8",
    "Conversations_Batch_9",
]

jupyter_gdrive_folder_ids = [
    "1Z1bdYMe2Qmo_vs-OaKDaYIiV3rIqLJH9", # V0
    "1sfPFHkXYpKyY41V0pfz3Qw3k4VLy5Hvb", # V1
    "1jV7WA5zB172DJUp7Z2XzHr62E6U6_NtY",
]

delivery_sheet_id = "1eUif5I8xhHU8fY0X9v8r2JI9hWPh7Dq_9VXpSIHwww4"
delivery_jsonl_gdrive_folder_id = "1wSCoCECyPgonCmBFSmNyMph71eJu18wx"
destination_folder_url = f"https://drive.google.com/drive/folders/{delivery_jsonl_gdrive_folder_id}"
DELIVERY_BATCH_NAME = "Batch 11"

insights_sheet_id = "1v_O33STdi_h7taPd3MkD0fiqRx7rqr_aAQWGnlOfr_w"
INSIGHTS_VERSION_TAB = "v1 (Jan 25)"

## Source Code


In [2]:
import sys 
sys.path.append('../../')
import io
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

import nbformat
import pandas as pd
from tqdm import tqdm
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload



def get_number_of_turns(messages):
    count = 0
    for message in messages:
        if message["role"] == "User":
            count += 1
    return count





def standardize_date_format(date):
    """
    Given a date string, standardize the date format to YYYY/MM/DD.
    """
    try:
        # Parse the date string into a datetime object
        standardized_date = datetime.strptime(date, "%Y/%m/%d")
    except ValueError:
        try:
            # Attempt to parse other common formats here
            # Example: MM/DD/YYYY
            standardized_date = datetime.strptime(date, "%m/%d/%Y")
        except ValueError:
            return "Invalid date format"

    # Format the datetime object into the desired string format
    return standardized_date.strftime("%Y/%m/%d")
###################################


#######################
# Tool Export #
#######################

import requests
import pandas as pd

def extract_tool_data(host="https://labeling.turing.com", api_key='44502e16-b4a7-49d3-90e1-d3d1fd698828', status='completed'):
    """
    Makes a GET request to the specified URL with the given headers,
    downloads the JSON response, and parses it into a pandas DataFrame.

    Parameters:
    - url (str): URL to make the GET request to.
    - headers (dict): Dictionary of headers to include in the request.

    Returns:
    - DataFrame: Pandas DataFrame containing the parsed JSON data.
    """

    # URL and headers for the request
    url = f"{host}/api/conversations/download-json?status={status}"
    headers = {
        'accept': 'application/json',
        'api-key': api_key
    }

    # Make the GET request
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a DataFrame
        data = response.json()
        df = pd.json_normalize(data)
        return df
    else:
        print(f"Failed to download data. Status code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure


def format_tool_export_for_sheet_parity(df):

    sheet_parity_columns = {
        'colabLink': 'task_link', 
        'seed.metadata.Project / Action': 'project_action', 
        'seed.metadata.Technical Domain': 'technical_domain', 
        'humanUser.turingEmail': 'assigned_to_email', 
        'status': 'completion_status', 
        'durationMinutes': 'duration_mins', 
        'completedAt': 'completion_date'
    }

    placeholder_columns = [
        "modified_question?",
        "comments",
        "metadata__type",
        "metadata__target_length",
        "review_status",
        "reviewer_email",
        "Start Time",
        "End Time"
    ]

    # Rename columns to align with sheet
    df = df.rename(columns=sheet_parity_columns)

    # Add placeholder columns
    for col in placeholder_columns:
        df[col] = None

    # Remove the detailed guide from the project_action column
    df['project_action'] = df['project_action'].str.replace(r'\[.*?\]\(.*?\)', '', regex=True)

    # Combine project_action and technical_domain into a single column
    df['metadata__topic'] = df['project_action'] + ' > ' + df['technical_domain']

    # For display purposes, converting datetime objects to 'MM/DD/YYYY' format strings
    df['completion_date'] = pd.to_datetime(df['completion_date']).dt.strftime('%m/%d/%Y')

    # Rename 'completed' status to 'Done'
    df['completion_status'] = df['completion_status'].replace('completed', 'Done')

    return df[["task_link", 'metadata__topic', "assigned_to_email", "completion_status", "duration_mins", "completion_date", "modified_question?", "comments", "metadata__type", "metadata__target_length", "review_status", "reviewer_email", "Start Time", "End Time"]]

# Example usage
df = extract_tool_data()
df = format_tool_export_for_sheet_parity(df)


#########################
    # Colab #
#########################


def update_colab_notebook(colab_link, local_nb_path, sa_creds_path):
    """
    Update a Google Colab notebook file in Google Drive.

    :param colab_link: The link to the Colab notebook in Google Drive.
    :param local_nb_path: The local path of the notebook file to upload.
    :param sa_creds_path: The path to the service account credentials.
    """
    try:
        # Extract file ID from the Colab link
        file_id = colab_link.split('/drive/')[1].split('/')[0]
    except IndexError:
        raise ValueError("Invalid Colab link format")

    # Initialize Google Drive API
    SCOPES = ['https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(sa_creds_path, scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    # Specify the file to upload
    media = MediaFileUpload(local_nb_path, resumable=True)

    # Update the file
    try:
        updated_file = service.files().update(fileId=file_id, media_body=media).execute()
        return f"Updated file ID: {updated_file.get('id')}"
    except Exception as e:
        return f"Error updating file: {e}"


def get_colab_notebook(colab_link, sa_creds_path) -> nbformat.NotebookNode:
    try:
        # Extract file ID from the Colab link
        file_id = colab_link.split('/drive/')[1].split('/')[0]
    except IndexError:
        raise ValueError("Invalid Colab link format")

    # Initialize Google Drive API
    SCOPES = ['https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(sa_creds_path, scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    # Download the file
    try:
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
        
        # Load as nbformat notebook
        notebook = nbformat.reads(fh.getvalue().decode(), as_version=4)
        return notebook
    except Exception as e:
        print(f"Error downloading file: {e}")
        return None


def get_file_name_from_colab_link(colab_link, service_account_file):
    try:
        file_id = colab_link.split('/drive/')[1]
    except IndexError:
        return None

    SCOPES = ['https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(service_account_file, scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    try:
        file = service.files().get(fileId=file_id).execute()
        return file.get('name')
    except Exception as e:
        return None


def fetch_file_names_parallel(links, service_account_file, max_workers=100):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(get_file_name_from_colab_link, link, service_account_file): link for link in links}
        results = {}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching File Names"):
            link = futures[future]
            try:
                file_name = future.result()
                results[link] = file_name
            except Exception as e:
                results[link] = None
        return results

## Read Remote Sheet

In [3]:
from src.sheets_utils import download_sheet_as_df


progress_batches = []
for sheet_name in included_sheet_names:
    print(sheet_name)
    bdf = download_sheet_as_df(service_account_file, tracking_sheet_id, sheet_name)
    progress_batches.append(bdf)

tool_df = extract_tool_data()
tool_df = format_tool_export_for_sheet_parity(tool_df)
progress_batches.append(tool_df)


df = pd.concat(progress_batches, ignore_index=True)
completed_df = df[df["completion_status"] == "Done"]
completed_df = completed_df.drop_duplicates(subset=["task_link"])

delivered = pd.concat([
    download_sheet_as_df(service_account_file, delivery_sheet_id, "Batch 5"),
    download_sheet_as_df(service_account_file, delivery_sheet_id, "Batch 6"),
    download_sheet_as_df(service_account_file, delivery_sheet_id, "Batch 7"),
    download_sheet_as_df(service_account_file, delivery_sheet_id, "Batch 8"),
    download_sheet_as_df(service_account_file, delivery_sheet_id, "Batch 9"),
    download_sheet_as_df(service_account_file, delivery_sheet_id, "Batch 10"),
], ignore_index=True)


completed_to_be_delivered_df = completed_df[~completed_df["task_link"].isin(delivered["task_link"])]
completed_to_be_delivered_df

Conversations_Batch_7
Conversations_Batch_8
Conversations_Batch_9


,task_link,metadata__topic,assigned_to_email,completion_status,modified_question?,duration_mins,completion_date,comments,metadata__type,metadata__target_length,review_status,reviewer_email,Start Time,End Time
505,5,python basics & scripting - explain complex co...,chandrashekhar.s@turing.com,Done,FALSE,45,7/2/2024,,,,Reviewed,paulo.c@turing.com,None,None
1249,https://colab.research.google.com/drive/1ISirh...,cloud_computing_OR_frameworks__write___modify_...,aarunik.g@turing.com,Done,,60,2/19/2024,None,None,None,None,None,None,None
1250,https://colab.research.google.com/drive/1ca91-...,cloud_computing_OR_frameworks__write___modify_...,aarunik.g@turing.com,Done,,60,2/19/2024,None,None,None,None,None,None,None
1395,https://colab.research.google.com/drive/1_slXe...,web_development__interview_prep__0_V8_A.ipynb,adil.m@turing.com,Done,,75,2/19/2024,None,None,None,None,None,None,None
1396,https://colab.research.google.com/drive/1uU28j...,web_development__interview_prep__4_V8_A.ipynb,adil.m@turing.com,Done,,70,2/19/2024,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761,https://colab.research.google.com/drive/1-T2Bx...,write code in python > problem solving,elsadek.a@turing.com,Done,None,40,02/22/2024,None,None,None,None,None,None,None
3762,https://colab.research.google.com/drive/1nvV4t...,write / modify / fix SQL code > data analysis,marcus.a@turing.com,Done,None,13,02/18/2024,None,None,None,None,None,None,None
3763,https://colab.research.google.com/drive/19BLNm...,write code in python > machine learning,elsadek.a@turing.com,Done,None,63,02/23/2024,None,None,None,None,None,None,None
3764,https://colab.research.google.com/drive/1CiGvf...,write code in python > problem solving,elsadek.a@turing.com,Done,None,40,02/23/2024,None,None,None,None,None,None,None


In [9]:
from src.llm_reviewer.notebook_parser import notebook_parser
from concurrent.futures import ThreadPoolExecutor, as_completed


notebooks = []
results = []
errors = []

def process_task_link(task_link):
    try:
        notebook = get_colab_notebook(task_link, service_account_file)
        parsed_notebook = notebook_parser(notebook)
        parsed_messages = parsed_notebook["messages"]
        number_of_turns = get_number_of_turns(parsed_messages)
        return parsed_notebook, {
            "task_link": task_link,
            "n_messages": len(parsed_messages),
            "number_of_turns": number_of_turns,
        }
    except Exception as e:

        return None, {
            "task_link": task_link,
            "error": str(e)
        }

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(process_task_link, task_link) for task_link in completed_to_be_delivered_df["task_link"].tolist()]
    for future in as_completed(futures):
        notebook, result = future.result()
        if notebook is not None:
            notebooks.append(notebook)
            results.append(result)
        else:
            errors.append(result)


results_df = pd.DataFrame(results)
results_df

Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='User', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='User', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='User', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompl

/home/joe96/projects/turing/character.ai/character_tasks/venv/lib/python3.9/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='User', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Assistant', role='assistant', function_call=None, tool_calls=None), logprobs=None)
Choice(finish_reason='stop', index=0, message

,task_link,n_messages,number_of_turns
0,https://colab.research.google.com/drive/1j1MYr...,8,4
1,https://colab.research.google.com/drive/1uU28j...,18,9
2,https://colab.research.google.com/drive/1_slXe...,8,4
3,https://colab.research.google.com/drive/1GmwZR...,16,8
4,https://colab.research.google.com/drive/1xtWX1...,10,5
...,...,...,...
664,https://colab.research.google.com/drive/1WPcWs...,7,4
665,https://colab.research.google.com/drive/18vLHs...,13,7
666,https://colab.research.google.com/drive/1CiGvf...,6,3
667,https://colab.research.google.com/drive/1gFo4j...,4,2


## GPT Review

In [7]:
import os
from typing import List
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import tiktoken
from pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI
from llama_index import ServiceContext, set_global_service_context
from llama_index.program import OpenAIPydanticProgram
from llama_index.callbacks import CallbackManager, TokenCountingHandler

api_key = os.environ["OPENAI_API_KEY"]


token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4-1106-preview").encode
)
callback_manager = CallbackManager([token_counter])

class Feedback(BaseModel):
    score: int = Field(description="A score representing how good the conversation is in the given quality aspect, 1 is terrible, 5 is exemplary and flawless.", ge=1, le=5)
    issues: List[str] = Field(description="A concrete list of issues in the conversation. 15 words or less each.")
    praises: List[str] = Field(description="A concrete list of praise for exceptional behavior the conversation. 15 words or less each.")


class QualityAspect(BaseModel):
    name: str = Field(description="The name of the quality aspect.")
    instruction: str = Field(description="Instructions & details on how to inspect this quality aspect.")


quality_aspects = {
    "Overall": [
        QualityAspect(
            name="Completness",
            instruction="""
            How complete is the conversation? Completeness is defined as:
            - The assistant always responds to the user.
            - The conversation contains at least 1 back and forth between the user and the assistant.
            - The conversation flow is not broken.

            JUDGE THE ENTIRE CONVERSATION AS A WHOLE.
            """
        ),
        QualityAspect(
            name="Project Scope Validity",
            instruction="""
            How much does the conversation align with the "Project / Action" value in metadata? 
            - Write code in python: this should have the user make requests that elicit python code writing behavior from the assistant.
            - Explain code: this should have the user present medium/high complexity code to the assistant and have the assistant explain it
            - Fix / refactor / optimize code: this should have the user present medium/high complexity code to the assistant and have the assistant do modifications on it as requested.
            - Debug error trace: the user should present a stack trace and some code and the assistant will find what the problem is and potentially fix the code (It's okay to have situations where the bug is not in the presented code but in a dependency... though this should be rare).... This EXCLUDES having the assistant teach the user how to use debug tools to find what the problem is themselves
            - Write unit tests: this should have the user present some low/medium/high complexity code to the assistant and have the assistant write tests for it... maximizing test coverage. (Critical Path first, Corner Cases Second)
            - Write CI/CD code: this should have the user request some help from the assistant in writing ci/cd pipelines in any flavor. (Github actions, Gitlab, Jenkins... etc)
            - Do a code review: this should have the user present some code snippet and request the assistant to review the code as if it's a PR... providing high level conceptual feedback, modifying any bugs and using inline comments to mark changes or suggest alternatives.
            - Write / modify / fix beam code: this should have the user present some data schema or dummy data and have the assistant write beam code for it.
            - Write / modify / fix spark code: this should have the user present some data schema or dummy data and have the assistant write spark code for it.
            - Write end to end ML training code: scenarios where the conversation has the user and assistant solving a problem e2e data eda/prep, feature extraction, training, maybe some evals and visuals as well
            - Help me take an interview: scenario where the user requests the assistant to act as an interviewer and do a mock interview with a focus on a certain area... this should also include some final section where the assistant gives feedback to the user on how to be better... etc (Take inspiration from real interview questions, they should be at least medium complexity and occasionally challenging)
            - Answer ML research questions: this is where the user will ask some cutting edge conceptual questions related to ML Research Hot topics to the assistant... assistant can but is not obligated to provide code as a response.
            - Answer infra questions: user asks some conceptual or code snippet related questions within the scope of cloud, backend, database, development tools... all flavors are welcome!
            - Write / modify / fix SQL code: this should have the user elicit interaction from the assistant within the context of SQL code.
            - Write / modify / fix JavaScript code: this should have the user elicit interaction from the assistant within the context of Javascript code.
            - Scrape a website: this should have the user present some html and the assistant write code to scrape it.

            JUDGE THE ENTIRE CONVERSATION AS A WHOLE.
            """
        ),
    ],
    "User": [
        QualityAspect(
            name="Natural & Realistic", 
            instruction="""
            How does the user interaction resemble a real conversation and interactions a real user would have with a highly intelligent coding assistant over chat.

            ONLY JUDGE THE USER MESSAGES. DO NOT JUDGE THE ASSISTANT MESSAGES.
            """
        )
    ],
    "Assistant": [
        QualityAspect(
            name="Accuracy", 
            instruction="""
            How good is the code that the assistant generates.
            Code Qualities:
            #   - Correctness
            #   - Optimality
            #   - PEP8 Compliance & Readability

            How good is the text that the assistant generates.
            Text Qualities:
            #   - Spelling
            #   - Grammar
            #   - Capitalization & Punctuation
            

            ONLY JUDGE THE ASSISTANT MESSAGES. DO NOT JUDGE THE USER MESSAGES.
            """
        ),
        QualityAspect(
            name="Consumability", 
            instruction="""
            - How good is the markdown formatting that the assistant generates. Is it leveraging markdown syntax tools to maximize the readability of the text?
            - Information Density (Should be a sweet spot leaning on the concise side, but not too concise... definitely not too verbose)
            - Explains Code Well by adding comments tailored for the user level assuming a beginner user by default

            ONLY JUDGE THE ASSISTANT MESSAGES. DO NOT JUDGE THE USER MESSAGES.
            """
        ),
        QualityAspect(
            name="Engageness",
            instruction="""
            - How engaging is the assistant's messages? Does it keep the user engaged and interested in the conversation?
            - Does the assistant ask questions to the user to keep the conversation going?

            ONLY JUDGE THE ASSISTANT MESSAGES. DO NOT JUDGE THE USER MESSAGES.
            """
        ),
        QualityAspect(
            name="Right level of detail",
            instruction="""
            - How concise is the assistant's messages? Does it keep the messages short and to the point?
            - In case there are too many points to cover, does the assistant prioritize, emphasize the most helpful ones and provide a summary at the end for the rest?
            - Does the assistant avoid providing unsolicited information/code that is not helpful to the user?

            ONLY JUDGE THE ASSISTANT MESSAGES. DO NOT JUDGE THE USER MESSAGES.
            """
        ),
    ]
}


def inspect_conversation_quality_aspect(conversation: List[List[dict]], quality_aspect: QualityAspect):
    """Inspect a conversation for a given quality aspect."""

    prompt_template_str = """
    IDENTITY:
    You are one of many specialized judges, so precisely focus on your quality aspect only.

    SITUATION:
    A large team is building a dataset of illustractions of dialogues showcasing the interaction between a user and a highly intelligent AI in the context of software development scenarios.
    - The user's replies should closely resemble authentic user engagement.
    - The AI's responses should aim to provide maximum benefit to the user.

    INSTRUCTIONS:
    Given the following conversation, please rate the quality of the conversation according to the given quality aspect.
    
    ALL QUALITY ASPECTS:
    {all_quality_aspects}

    YOUR QUALITY ASPECT:
    {quality_aspect}
    
    CONVERSATION:
    {conversation}
    """
    program = OpenAIPydanticProgram.from_defaults(
        llm=OpenAI(api_key=api_key, model="gpt-4-1106-preview", temperature=0),
        callback_manager=callback_manager,
        output_cls=Feedback, 
        prompt_template_str=prompt_template_str, 
    )
    all_quality_aspects = "\n".join([f"- {key}: {quality_aspect.name}" for key in quality_aspects.keys() for quality_aspect in quality_aspects[key]])
    output = program( 
        all_quality_aspects=all_quality_aspects,
        quality_aspect=quality_aspect.model_dump(),
        conversation=conversation,
        description="Judge the quality of the conversation according to the given quality aspect. Provide constructive criticism, rarely praise."
    )
    return output


def inspect_all_conversation_quality_aspects(conversation) -> dict:
    """Inspect a conversation for all quality aspects."""

    quality_results = {}
    for key in quality_aspects.keys():
        for quality_aspect in quality_aspects[key]:
            r = inspect_conversation_quality_aspect(conversation, quality_aspect)
            quality_results[f"{key} - {quality_aspect.name}"] = r.model_dump()

    return quality_results

In [9]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

reviewed_results = []

def process_notebook(result, link):
    if result is None or len(result["messages"]) == 0:
        return None
    result["quality_review"] = inspect_all_conversation_quality_aspects(result)
    result["task_link"] = link
    return result

with tqdm(total=len(results), desc="Processing notebooks") as pbar:
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(process_notebook, result, link) for result, link in zip(notebooks, results_df["task_link"].tolist())]
        for future in as_completed(futures):
            r = future.result()
            if r is not None:
                reviewed_results.append(r)
            pbar.update(1)

Processing notebooks:   0%|          | 0/661 [00:00<?, ?it/s]

Processing notebooks: 100%|██████████| 661/661 [16:51<00:00,  1.53s/it]


In [15]:
data_skeleton = []
for rr in reviewed_results:
    scores = []
    feedback = ""
    for key in rr["quality_review"].keys():
        scores.append(rr["quality_review"][key]["score"])
        stringified_issues = "\n".join([f"- {issue}" for issue in rr["quality_review"][key]["issues"]])
        feedback += f"{key}: \n{stringified_issues}\n\n"

    data_skeleton.append({
        "task_link": rr["task_link"],
        "avg_score": sum(scores) / len(scores),
        "min_score": min(scores),
        "issues": feedback,
    })

df_gpt_reviews = pd.DataFrame(data_skeleton)


borderline_avg_flags = df_gpt_reviews.sort_values(by="avg_score", ascending=False)[df_gpt_reviews["avg_score"] < 4.3]
critical_mistake_flags = df_gpt_reviews.sort_values(by="min_score", ascending=False)[df_gpt_reviews["min_score"] < 3]

all_flags = pd.concat([borderline_avg_flags, critical_mistake_flags], ignore_index=True)
all_flags = all_flags.drop_duplicates(subset=["task_link"])


all_flags = all_flags.merge(completed_to_be_delivered_df, on="task_link", how="left")[["task_link", "avg_score", "min_score", "issues", "assigned_to_email"]]
all_flags = all_flags.rename(columns={"assigned_to_email": "original_author_email"})
all_flags["resolved_by_email"] = ""
all_flags["status"] = "Unclaimed"
all_flags["duration_mins"] = ""
all_flags["corrections"] = ""


from src.sheets_utils import upload_df_to_sheet, GoogleSheetsService

sheets_client = GoogleSheetsService(service_account_file, ['https://www.googleapis.com/auth/spreadsheets'])
sheets_client.ensure_sheet_exists(tracking_sheet_id, "historical__gpt_flags_3")
upload_df_to_sheet(service_account_file, tracking_sheet_id, "historical__gpt_flags_3", all_flags)


flagged_links = set(all_flags["task_link"].tolist())

results_df = results_df[~results_df["task_link"].isin(flagged_links)]

# Filter results df rows so that task_link column only contains valid colab links
results_df = results_df[results_df["task_link"].str.contains("https://colab.research.google.com/")]

nbs = []
for nb in notebooks:
    if "task_link" not in nb:
        continue

    if nb["task_link"] not in flagged_links:
        nbs.append(nb)


# Filter results_df so that task_link is in nbs.task_link
results_df = results_df[results_df["task_link"].isin([nb["task_link"] for nb in nbs])]
notebooks = nbs


results_df.shape, len(nbs)

,task_link,avg_score,min_score,issues
0,https://colab.research.google.com/drive/1Jv96_...,4.666667,4,Overall - Completness: \n\n\nOverall - Project...
1,https://colab.research.google.com/drive/1GmwZR...,4.666667,4,Overall - Completness: \n\n\nOverall - Project...
2,https://colab.research.google.com/drive/1lBqPQ...,4.500000,4,Overall - Completness: \n\n\nOverall - Project...
3,https://colab.research.google.com/drive/1F8pDE...,4.666667,4,Overall - Completness: \n\n\nOverall - Project...
4,https://colab.research.google.com/drive/1j1MYr...,4.833333,4,Overall - Completness: \n\n\nOverall - Project...
...,...,...,...,...
655,https://colab.research.google.com/drive/1S_Abj...,4.666667,4,Overall - Completness: \n\n\nOverall - Project...
656,https://colab.research.google.com/drive/1-T2Bx...,4.833333,4,Overall - Completness: \n\n\nOverall - Project...
657,https://colab.research.google.com/drive/1FgXO0...,4.666667,4,Overall - Completness: \n\n\nOverall - Project...
658,https://colab.research.google.com/drive/1CiGvf...,4.666667,4,Overall - Completness: \n\n\nOverall - Project...


## Extract Advanced Metadata

In [35]:
import os
from typing import List
from pydantic import BaseModel, Field
from llama_index.program import OpenAIPydanticProgram
from llama_index.llms.openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures

from tqdm import tqdm
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
api_key = os.environ["OPENAI_API_KEY"]


class HierarchicalCategory(BaseModel):
    """Data model for hierarchical category classification."""
    top_level: str
    sub_level: str

def classify_conversation_by_domain(conversation: List[dict]) -> HierarchicalCategory:
    DOMAIN_CATEGORIES = """
        - Python basics & scripting
        - Problem Solving
        - Interview Prep
        - Web Development
        - Testing
        - Cloud Computing / Frameworks
        - Data Analysis
        - Machine Learning
        - Other languages
        - Other
    """

    prompt_template_str = """
    Categorize the theme of user requests in the following conversation by domain into one of the following top-level categories, then sub categories that you think is descriptive & appropriate:
    {categories}

    Conversation:
    {conversation}
    """

    program = OpenAIPydanticProgram.from_defaults(
        llm=OpenAI(api_key=api_key, model="gpt-4-1106-preview", temperature=0),
        output_cls=HierarchicalCategory,
        prompt_template_str=prompt_template_str,
        verbose=False,
    )
    output = program(
        categories=DOMAIN_CATEGORIES,
        conversation=conversation["messages"],
    )
    return output


def classify_conversation_by_action(
    conversation: List[dict]
) -> HierarchicalCategory:
    ACTION_CATEGORIES = """
    - Write code in python: this should have the user make requests that elicit python code writing behavior from the assistant.
    - Explain code: this should have the user present medium/high complexity code to the assistant and have the assistant explain it
    - Fix / refactor / optimize code: this should have the user present medium/high complexity code to the assistant and have the assistant do modifications on it as requested.
    - Debug error trace: the user should present a stack trace and some code and the assistant will find what the problem is and potentially fix the code (It's okay to have situations where the bug is not in the presented code but in a dependency... though this should be rare).... This EXCLUDES having the assistant teach the user how to use debug tools to find what the problem is themselves
    - Write unit tests: this should have the user present some low/medium/high complexity code to the assistant and have the assistant write tests for it... maximizing test coverage. (Critical Path first, Corner Cases Second)
    - Write CI/CD code: this should have the user request some help from the assistant in writing ci/cd pipelines in any flavor. (Github actions, Gitlab, Jenkins... etc)
    - Do a code review: this should have the user present some code snippet and request the assistant to review the code as if it's a PR... providing high level conceptual feedback, modifying any bugs and using inline comments to mark changes or suggest alternatives.
    - Write / modify / fix beam code: this should have the user present some data schema or dummy data and have the assistant write beam code for it.
    - Write / modify / fix spark code: this should have the user present some data schema or dummy data and have the assistant write spark code for it.
    - Write end to end ML training code: scenarios where the conversation has the user and assistant solving a problem e2e data eda/prep, feature extraction, training, maybe some evals and visuals as well
    - Help me take an interview: scenario where the user requests the assistant to act as an interviewer and do a mock interview with a focus on a certain area... this should also include some final section where the assistant gives feedback to the user on how to be better... etc (Take inspiration from real interview questions, they should be at least medium complexity and occasionally challenging)
    - Answer ML research questions: this is where the user will ask some cutting edge conceptual questions related to ML Research Hot topics to the assistant... assistant can but is not obligated to provide code as a response.
    - Answer infra questions: user asks some conceptual or code snippet related questions within the scope of cloud, backend, database, development tools... all flavors are welcome!
    - Write / modify / fix SQL code: this should have the user elicit interaction from the assistant within the context of SQL code.
    - Write / modify / fix JavaScript code: this should have the user elicit interaction from the assistant within the context of Javascript code.
    - Scrape a website: this should have the user present some html and the assistant write code to scrape it.
    """
    prompt_template_str = """
    Categorize the user requests in the following conversation by requested action into one of the following top-level categories. Sub-level should be empty string always. In case there's no natural fit, use "Other" as the top-level category.
    
    Please note that there are "metadata" fields in the conversation that describe the intended top-level category via "Project / Action"... this should be considered, but may be overridden if the conversation is clearly about something else.

    Categories:
    {categories}

    Conversation:
    {conversation}

    """

    program = OpenAIPydanticProgram.from_defaults(
        llm=OpenAI(api_key=api_key, model="gpt-4-1106-preview", temperature=0),
        output_cls=HierarchicalCategory,
        prompt_template_str=prompt_template_str,
        verbose=False,
    )
    output = program(
        categories=ACTION_CATEGORIES,
        conversation=conversation,
    )
    return output


class SummaryResult(BaseModel):
    """Data model for the summary result."""
    summary: str = Field(
        description="A short summary containing 1 sentence, 15 words max, focused on the specific theme. [super concise language]"
    )

def exec_summary(conversation: List[List[dict]]):
    prompt_template_str = """
    Given the following conversation, please, generate an executive summary of the conversation.

    User Use Case, why user uses the Assistant in this conversation, in general terms, **for what** the User is using it. Not from a technical perspective, but from a daily life situation perspective. 
    Example: work, homework, exam, studying, inteview, debugging, etc...

    It should also contain a little bit of the context of the conversation, and the main goal of the conversation.

    Conversation:
    {conversation}
    """
    program = OpenAIPydanticProgram.from_defaults(
        llm=OpenAI(api_key=api_key, model="gpt-4-1106-preview", temperature=0),
        output_cls=SummaryResult,
        prompt_template_str=prompt_template_str,
        verbose=False,
    )
    output = program(
        conversation=conversation["messages"]
    )
    return output


class GPTEstimationResult(BaseModel):
    """Data model for the GPT estimation result."""
    estimated_duration: int = Field(
        description="The estimated duration of the conversation in minutes."
    )

def gpt_estimated_duration(conversation: List[dict]) -> int:
    prompt_template_str = """
    Given the following conversation which has been generated by a median skilled technical human playing both User and Assistant... He also is responsible for making sure the assistant responses are flawless...
    Estimate how many minutes it would take to Design, Write & Verify this (Conversation Length, Complexity).

    If you get this right, you will save my life.

    Conversation:
    {conversation}
    """
    program = OpenAIPydanticProgram.from_defaults(
        llm=OpenAI(api_key=api_key, model="gpt-4-1106-preview", temperature=0),
        output_cls=GPTEstimationResult,
        prompt_template_str=prompt_template_str,
        verbose=False,
    )
    output = program(
        conversation=conversation["messages"]
    )
    return output


def process_conversation__metadata_extraction(conversation, task_link):
    domain = classify_conversation_by_domain(conversation)
    action = classify_conversation_by_action(conversation)
    summary = exec_summary(conversation)
    estimate_duration = gpt_estimated_duration(conversation)
    conversation["metadata"].update({
        "domain": domain.model_dump(), 
        "action": action.model_dump()["top_level"], 
        "use_case_summary": summary.model_dump()["summary"],
        "gpt_estimated_duration": estimate_duration.model_dump()["estimated_duration"],
        "task_link": task_link
    })
    return conversation
    
    

def extract_metadata_parallel(conversations, task_links, max_workers=15):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_conversation__metadata_extraction, conversation, task_link)
            for conversation, task_link in zip(conversations, task_links)
        ]
        progress_bar = tqdm(total=len(futures))
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
            progress_bar.update(1)
        progress_bar.close()
    return results


metadata_results = extract_metadata_parallel(notebooks, results_df["task_link"].tolist())

100%|██████████| 644/644 [09:55<00:00,  1.08it/s]


In [ ]:
rich_task_metadata = []
for task_link in metadata_results:
    row_main = {
        "task_link": task_link["metadata"]["task_link"],
        "number_of_turns": get_number_of_turns(task_link["messages"]),
        "batch_id": DELIVERY_BATCH_NAME.split(" ")[-1],
        "domain": {
            "top_level": task_link["metadata"]["domain"]["top_level"],
            "sub_level": task_link["metadata"]["domain"]["sub_level"],
        },
        "action": task_link["metadata"]["action"],
        "use_case__summary": task_link["metadata"]["use_case_summary"],
        "gpt_estimated_duration": task_link["metadata"]["gpt_estimated_duration"],
    }
    rich_task_metadata.append(row_main)


rich_flattened_metadata = []
for metadata in rich_task_metadata:
    current_metadata = metadata.copy()
    for key in metadata.keys():
        if isinstance(metadata[key], dict):
            for sub_key in metadata[key].keys():
                current_metadata[f"{key}__{sub_key}"] = current_metadata[key][sub_key]
            current_metadata.pop(key)
    rich_flattened_metadata.append(current_metadata)

df_metadata__output = pd.DataFrame(rich_flattened_metadata)

In [ ]:
# projects_to_exclude = ["Scrape a website", "Help me take an interview", "Write / modify / fix beam code", "Write / modify / fix spark code"]

# historical_redo = completed_to_be_delivered_df.merge(df_metadata__output[df_metadata__output["action"].isin(projects_to_exclude)], on="task_link", how="inner")
# historical_redo = historical_redo.drop_duplicates(subset=["task_link"])
# historical_redo = historical_redo.rename(columns={"assigned_to_email": "original_author", "completion_date": "original_date"})
# historical_redo["resolved_by_email"] = ""
# historical_redo["completion_status"] = "Unclaimed"
# historical_redo["resolution_duration"] = ""
# historical_redo["completion_date"] = ""
# historical_redo["corrections"] = ""
# historical_redo[["task_link", "original_author", "action", "resolved_by_email", "resolution_duration", "completion_status", "completion_date", "corrections"]]

# from src.sheets_utils import upload_df_to_sheet, GoogleSheetsService

# sheets_client = GoogleSheetsService(service_account_file, ['https://www.googleapis.com/auth/spreadsheets'])
# sheets_client.ensure_sheet_exists(tracking_sheet_id, "historical__corrections_2")
# upload_df_to_sheet(service_account_file, tracking_sheet_id, "historical__corrections_2", historical_redo[["task_link", "original_author", "action", "resolved_by_email", "resolution_duration", "completion_status", "completion_date", "corrections"]])

In [39]:
from src.sheets_utils import upload_df_to_sheet, GoogleSheetsService

sheets_client = GoogleSheetsService(service_account_file, ['https://www.googleapis.com/auth/spreadsheets'])
df_metadata__output = df_metadata__output[["task_link", "batch_id", "number_of_turns", "gpt_estimated_duration", "action", "domain__top_level", "domain__sub_level", "use_case__summary"]]
values = df_metadata__output.values.tolist() # [df_metadata__output.columns.tolist()] + 
sheets_client.update_or_append_data_to_sheet(insights_sheet_id, INSIGHTS_VERSION_TAB, values)

Updated or appended data to 'v1 (Jan 25)'


{'spreadsheetId': '1v_O33STdi_h7taPd3MkD0fiqRx7rqr_aAQWGnlOfr_w',
 'tableRange': "'v1 (Jan 25)'!A1:H6148",
 'updates': {'spreadsheetId': '1v_O33STdi_h7taPd3MkD0fiqRx7rqr_aAQWGnlOfr_w',
  'updatedRange': "'v1 (Jan 25)'!A6149:H6792",
  'updatedRows': 644,
  'updatedColumns': 8,
  'updatedCells': 5152}}

## Upload JSONL

In [20]:
# Download insights mining & filter for this batch
insights_df = download_sheet_as_df(service_account_file, insights_sheet_id, INSIGHTS_VERSION_TAB)
current_batch = insights_df[insights_df["batch_id"] == DELIVERY_BATCH_NAME.split(" ")[-1]]

# Use the links to filter the results & notebooks
nbs = []
rs = []
current_batch_links = set(current_batch["task_link"].tolist())
for r, nb in zip(results, notebooks):
    if "task_link" not in r:
        continue
    if r["task_link"] in current_batch_links:
        nbs.append(nb)
        rs.append(r)

results_df = pd.DataFrame(rs)

# Show lengths for consistency check
len(nbs), len(rs), results_df.shape

(644, 644, (644, 3))

In [24]:
rich_task_metadata = []
for task_link in insights_df.to_dict(orient="records"):
    row_main = {
        "task_link": task_link["task_link"],
        "number_of_turns": task_link["number_of_turns"],
        "batch_id": task_link["number_of_turns"],
        "domain": {
            "top_level": task_link["domain__top_level"],
            "sub_level": task_link["domain__sub_level"],
        },
        "action": task_link["action"],
        "use_case__summary": task_link["use_case__summary"],
        "gpt_estimated_duration": task_link["gpt_estimated_duration"],
    }
    rich_task_metadata.append(row_main)

# Filter for same task links as rs
rich_task_metadata = [r for r in rich_task_metadata if r["task_link"] in results_df["task_link"].tolist()]
len(rich_task_metadata)

644

In [25]:
import os
import json

directory = f"json_conversations/{DELIVERY_BATCH_NAME}"
if not os.path.exists(directory):
    os.makedirs(directory)

valid_notebooks = []
for r, n in zip(rs, nbs):
    if r is None or r["n_messages"] == 0:
        continue
    n["task_link"] = r["task_link"]
    valid_notebooks.append(n)

parsed_jsons = []
for vn in valid_notebooks:
    for rtm in rich_task_metadata:
        if vn["task_link"] == rtm["task_link"]:
            vn["metadata"] = rtm
            parsed_jsons.append(vn)

for pj in parsed_jsons:
    pj["id"] = pj.pop("task_link").split("/")[-1]
    try:
        pj["metadata"].pop("duration_mins")
        pj["metadata"].pop("batch_id")
    except KeyError:
        pass

for i, conversation in enumerate(parsed_jsons):
    drive_id = conversation["id"] 
    with open(f"json_conversations/{DELIVERY_BATCH_NAME}/{drive_id}.json", "w") as f:
        f.write(json.dumps(conversation))

In [29]:
import json

from src.gdrive_api import build_service
from src.gdrive_api.folder_upload import upload_folder

from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

import io
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload, MediaIoBaseUpload



def upload_gdrive_file(file_contents, folder_id, service_account_file):
    """
    Re-uploads a dictionary from memory as a JSON file to Google Drive. 

    Parameters:
    - file_contents: dict
        The file contents to upload.
    - folder_id: str
        The ID of the file to upload.
    - service_account_file: str
        The path to the service account file.

    Returns True if the file was successfully uploaded, False otherwise.
    """
    # Initialize Google Drive API
    SCOPES = ['https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(service_account_file, scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    # Convert the dictionary to JSON and prepare it for upload
    file_metadata = {
        'name': f'{file_contents["id"]}.json',
        'parents': [folder_id]
    }
    file_data = io.BytesIO(json.dumps(file_contents).encode('utf-8'))
    media = MediaIoBaseUpload(file_data, mimetype='application/json')

    # Upload the file
    try:
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        return True
    except Exception as e:
        print(f"An error occurred: {e}")
        return False


def parallel_execute_with_progress(function, arguments, max_workers=10):
    """
    Executes a function in parallel with multiple arguments displaying a tqdm progress bar.

    Parameters
    function: function
        The function to execute
    arguments: list
        A list of tuples, where each tuple contains the arguments to pass to the function
    max_workers: int
        The maximum number of workers to use

    Returns a list of results
    """
    # Create a thread pool
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Execute the function for each argument
        futures = [executor.submit(function, *args) for args in arguments]
        results = []
        for future in tqdm(as_completed(futures), total=len(futures)):
            results.append(future.result())
    return results


statuses = parallel_execute_with_progress(upload_gdrive_file, [(pj, delivery_jsonl_gdrive_folder_id, service_account_file) for pj in parsed_jsons], max_workers=50)
pd.Series(statuses).value_counts()

100%|██████████| 644/644 [00:50<00:00, 12.70it/s]


True    644
Name: count, dtype: int64

In [30]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Function to check if a file is a folder
def is_folder(file):
    return file.get('mimeType') == 'application/vnd.google-apps.folder'

# Function to process files and folders
def process_files(service, folder_id, parent_folders=[]):
    query = f"'{folder_id}' in parents and trashed = false"
    page_token = None

    all_files = []
    while True:
        response = service.files().list(q=query,
                                        spaces='drive',
                                        fields='nextPageToken, files(id, name, mimeType, webViewLink)',
                                        pageToken=page_token).execute()

        for file in response.get('files', []):
            # Skip 'tool_data' folder
            if file.get('name') == 'tool_data' and is_folder(file):
                continue

            all_files.append(file)

            # Process the file or folder
            print('Processing:', '/'.join(parent_folders + [file.get('name')]))

            # If it's a folder, recursively process its contents
            if is_folder(file):
                children_files = process_files(service, file.get('id'), parent_folders + [file.get('name')])
                all_files.extend(children_files)

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Authenticate and create the service
SERVICE_ACCOUNT_FILE = service_account_file
SCOPES = ['https://www.googleapis.com/auth/drive']
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

# Replace with your Google Drive folder ID
folder_id = destination_folder_url.split("/")[-1]

# Start processing from the specified folder
all_files = process_files(service, folder_id)

jsonl_df = pd.DataFrame(all_files)
jsonl_df = jsonl_df[["id", "name", "webViewLink"]]
jsonl_df

Processing: 1-T2Bxssul-cYzS8QdPazVA_IIYW_8JW2.json
Processing: 1CiGvfxnTRuaj_Vz0YFRlkugt2aCaInY3.json
Processing: 1gFo4j12t_J-ldX7AfpsPn0MtJzJJeHH8.json
Processing: 18vLHs0ouk3_3CW1Ouw1zImSdjpkF4EE_.json
Processing: 19BLNm2kR9hRGNTRGVJQo1WfWB6JW8yUO.json
Processing: 1nvV4tp4wXVbE1BPolqmtAzD2orQWP4lC.json
Processing: 110weswsFPeMcW00rmj0E_8RqUG4gSLF-.json
Processing: 13qB6bBJ_rhK5XI7S9WxRn-PmGPvxvnd5.json
Processing: 1UIHktx7SxW0d5S2FCsctw55yzvZyaZO2.json
Processing: 1S_AbjzWfx6eldeajDjSJAoNL1b4IOOeb.json
Processing: 1WPcWsi5FaHY30NPNH_Et5gRThCvSA6Wb.json
Processing: 1FgXO0GsR0tMjeuLHEOx1NNMKpxhnmTxG.json
Processing: 1Fmnj3E6zQENwXSoQvC2oyIEdovH9NymG.json
Processing: 1NgJKJvq00pgU2u15zoWVWFoVtXKah-8R.json
Processing: 1PcrMQyk_dQas3MQ1rdeIXxGygvTj94lS.json
Processing: 1nwqMoMcgwo72lJeYVNrfJ0vA9ueAE2iS.json
Processing: 1AA1CGUIlTU2MPCnexbtulJt1ODRUytPi.json
Processing: 1B8kWjDIzwkX-Qk69miyUq63euklcnD3O.json
Processing: 1nTiQAwJib1DxLW-NfpuQ1z92iTxSIK2k.json
Processing: 1g_IiE55IhIcG1moNRw

,id,name,webViewLink
0,1gDHNp-3SGT9jv9v7IFhIk0QJ0BXXJLwn,1-T2Bxssul-cYzS8QdPazVA_IIYW_8JW2.json,https://drive.google.com/file/d/1gDHNp-3SGT9jv...
1,1HWsDfkGwIqFJd3HGBkKMQeWgIM25sq6y,1CiGvfxnTRuaj_Vz0YFRlkugt2aCaInY3.json,https://drive.google.com/file/d/1HWsDfkGwIqFJd...
2,1sjtNn_FpTmh4Y_NwQoovcJ1NIIQztwF-,1gFo4j12t_J-ldX7AfpsPn0MtJzJJeHH8.json,https://drive.google.com/file/d/1sjtNn_FpTmh4Y...
3,1wyX7M98j0CJW0VuuogbJIjkmlLYTMio6,18vLHs0ouk3_3CW1Ouw1zImSdjpkF4EE_.json,https://drive.google.com/file/d/1wyX7M98j0CJW0...
4,1w1DSLeLKvd2FBpECMspj79wMQQqbs5qA,19BLNm2kR9hRGNTRGVJQo1WfWB6JW8yUO.json,https://drive.google.com/file/d/1w1DSLeLKvd2FB...
...,...,...,...
639,1Cn7ajO_CZYXyI8g_RPx7NIjXlucCwSfH,1e6ORvNVBNteC_5q2IhnfdReEdsn_bb_-.json,https://drive.google.com/file/d/1Cn7ajO_CZYXyI...
640,1Ay0oht_Viu2IRvycMOezP92DBfid9Ltq,1ca91-ZE2u1uigCJgT-da-fypBEtzYNvB.json,https://drive.google.com/file/d/1Ay0oht_Viu2IR...
641,1FpcZv93zjZeATjXZWiGMxDrWz0xBp6l4,1-lWGDxx41n9sg87yIsqVTur1ZVpR0nSO.json,https://drive.google.com/file/d/1FpcZv93zjZeAT...
642,1EuRDw6xBrAxbK3c8MLURLZDXv82-6-FG,15eYkWuWKMfAJF5yntXj7FB-0fVdjes6s.json,https://drive.google.com/file/d/1EuRDw6xBrAxbK...


In [31]:
parsed_jsons_ref = [
    {
        "colab_id": pj["id"],
        "task_link": pj["metadata"]["task_link"],
        "number_of_turns": pj["metadata"]["number_of_turns"],
    }
    for pj
    in parsed_jsons
]
conversation_df = pd.DataFrame(parsed_jsons_ref)
conversation_df

,colab_id,task_link,number_of_turns
0,1j1MYrC5QM0lMjKqyCdEzyZZEEotFjl-f,https://colab.research.google.com/drive/1j1MYr...,4
1,1uU28jLtLHaZM63YQf2HAUqlHm76Y84Uu,https://colab.research.google.com/drive/1uU28j...,9
2,1_slXeCkFcpl5tT_bh1SnFzIz3qPSNQjW,https://colab.research.google.com/drive/1_slXe...,4
3,1GmwZRUz9Owkf2TclEkiiq3s58YCekLnd,https://colab.research.google.com/drive/1GmwZR...,8
4,1xtWX10-Qg6e0EHglgMpDWp0NARwIisYH,https://colab.research.google.com/drive/1xtWX1...,5
...,...,...,...
639,1WPcWsi5FaHY30NPNH_Et5gRThCvSA6Wb,https://colab.research.google.com/drive/1WPcWs...,3
640,18vLHs0ouk3_3CW1Ouw1zImSdjpkF4EE_,https://colab.research.google.com/drive/18vLHs...,7
641,1CiGvfxnTRuaj_Vz0YFRlkugt2aCaInY3,https://colab.research.google.com/drive/1CiGvf...,3
642,1gFo4j12t_J-ldX7AfpsPn0MtJzJJeHH8,https://colab.research.google.com/drive/1gFo4j...,2


In [32]:
jsonl_df["colab_id"] = jsonl_df["name"].apply(lambda x: x.split(".")[0])


df_merged = conversation_df.merge(jsonl_df, on="colab_id", how="inner")
df_merged = df_merged[["task_link", "number_of_turns", "webViewLink"]]
df_merged = df_merged.rename(columns={"webViewLink": "jsonl_link"})
df_merged

,task_link,number_of_turns,jsonl_link
0,https://colab.research.google.com/drive/1j1MYr...,4,https://drive.google.com/file/d/1JDiCGG78xDtjP...
1,https://colab.research.google.com/drive/1uU28j...,9,https://drive.google.com/file/d/1x5Y-4MEakvBaE...
2,https://colab.research.google.com/drive/1_slXe...,4,https://drive.google.com/file/d/1Lopn3R4e4_dgM...
3,https://colab.research.google.com/drive/1GmwZR...,8,https://drive.google.com/file/d/1x2oHoNUXg0R4i...
4,https://colab.research.google.com/drive/1xtWX1...,5,https://drive.google.com/file/d/1zr6CWZ5nxrAxw...
...,...,...,...
639,https://colab.research.google.com/drive/1WPcWs...,3,https://drive.google.com/file/d/1UbPmxmbDtbHII...
640,https://colab.research.google.com/drive/18vLHs...,7,https://drive.google.com/file/d/1wyX7M98j0CJW0...
641,https://colab.research.google.com/drive/1CiGvf...,3,https://drive.google.com/file/d/1HWsDfkGwIqFJd...
642,https://colab.research.google.com/drive/1gFo4j...,2,https://drive.google.com/file/d/1sjtNn_FpTmh4Y...


## Upload Batch Sheet

In [33]:
from src.sheets_utils import upload_df_to_sheet, GoogleSheetsService

cols = ["task_link", "jsonl_link", "number_of_turns"]

sheets_client = GoogleSheetsService(service_account_file, ['https://www.googleapis.com/auth/spreadsheets'])
sheets_client.ensure_sheet_exists(delivery_sheet_id, DELIVERY_BATCH_NAME)
upload_df_to_sheet(service_account_file, delivery_sheet_id, DELIVERY_BATCH_NAME, df_merged[cols])

Created new sheet: 'Batch 11'
